In [1]:
# Cell 1 — Imports + Config
import sys, os, torch, torch.nn as nn, torch.optim as optim
import numpy as np, h5py, matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm as tqdm_nb
from datetime import datetime
import warnings; warnings.filterwarnings("ignore")
from src.config import TrainingConfig

config    = TrainingConfig()
device    = config.DEVICE
LOAD_MODE = config.DATA_MODE   # ← from .env, no manual edit needed

print("=" * 70)
print("NOTEBOOK 04 — MODEL ARCHITECTURE")
print("=" * 70)
config.print_summary()

NOTEBOOK 04 — MODEL ARCHITECTURE

🚀 CIVICPULSE TRAINING CONFIGURATION (Benchmark-Optimized)
Device           : cpu
VRAM Available   : 0.0 GB
Batch Size       : 32
Data Mode        : normal
Patch Size       : 256×256 cells
ConvLSTM         : 64 hidden, 2 layers
Learning Rate    : 0.001
Sequence Length  : 4 timesteps



In [2]:
# Cell 2 — ConvLSTM Cell
class ConvLSTMCell(nn.Module):
    def __init__(self, in_channels, hidden_channels, kernel_size=3):
        super().__init__()
        self.hidden_channels = hidden_channels
        padding = kernel_size // 2
        self.conv_gates = nn.Conv2d(
            in_channels + hidden_channels, 2 * hidden_channels, kernel_size, padding=padding)
        self.conv_candidate = nn.Conv2d(
            in_channels + hidden_channels, hidden_channels, kernel_size, padding=padding)

    def forward(self, inputs, hidden_state):
        h, c = hidden_state
        combined = torch.cat([inputs, h], dim=1)
        gates = self.conv_gates(combined)
        reset_gate, update_gate = torch.split(gates, self.hidden_channels, dim=1)
        reset_gate  = torch.sigmoid(reset_gate)
        update_gate = torch.sigmoid(update_gate)
        combined_candidate = torch.cat([inputs, reset_gate * h], dim=1)
        candidate = torch.tanh(self.conv_candidate(combined_candidate))
        new_c = (1 - update_gate) * c + update_gate * candidate
        new_h = torch.tanh(new_c) * update_gate + (1 - update_gate) * h
        return new_h, new_c

In [3]:
# Cell 3 — ConvLSTM Encoder-Decoder
class ConvLSTMEncoderDecoder(nn.Module):
    def __init__(self, in_channels=1,
                 hidden_channels=None, num_layers=None, kernel_size=3):
        super().__init__()
        # Pull from config if not explicitly passed
        _cfg = TrainingConfig()
        hidden_channels = hidden_channels or _cfg.HIDDEN_CHANNELS
        num_layers      = num_layers      or _cfg.NUM_LAYERS
        self.hidden_channels = hidden_channels
        self.num_layers      = num_layers
        self.encoder_cells = nn.ModuleList([
            ConvLSTMCell(in_channels if i == 0 else hidden_channels,
                         hidden_channels, kernel_size)
            for i in range(num_layers)])
        self.decoder_cells = nn.ModuleList([
            ConvLSTMCell(hidden_channels, hidden_channels, kernel_size)
            for _ in range(num_layers)])
        self.output_conv = nn.Conv2d(hidden_channels, 1, kernel_size=1)

    def forward(self, x):
        B, T, C, H, W = x.shape
        h = [torch.zeros(B, self.hidden_channels, H, W, device=x.device, dtype=x.dtype)
             for _ in range(self.num_layers)]
        c = [torch.zeros(B, self.hidden_channels, H, W, device=x.device, dtype=x.dtype)
             for _ in range(self.num_layers)]
        for t in range(T):
            xt = x[:, t]
            for layer in range(self.num_layers):
                h[layer], c[layer] = self.encoder_cells[layer](
                    xt if layer == 0 else h[layer-1], (h[layer], c[layer]))
        for layer in range(self.num_layers):
            inp = h[layer-1] if layer > 0 else h[0]
            h[layer], c[layer] = self.decoder_cells[layer](inp, (h[layer], c[layer]))
        return self.output_conv(h[-1])   # (B, 1, H, W)

In [4]:
# Cell 4 — Load Test Data (mode-aware)
h5_path = "data/processed/india_sample.h5"

if LOAD_MODE == "hdf5":
    print("📂 HDF5 mode: lazy-loading patch for test...")
    with h5py.File(h5_path, "r") as h5:
        data = h5["population_data"][:, :256, :256]
    print(f"  Loaded patch: {data.shape}")
else:
    print("📂 Normal mode: full numpy load...")
    tel_seq  = np.load("data/processed/telangana_population_sequence.npy")
    maha_seq = np.load("data/processed/maharashtra_population_sequence.npy")
    T, H1, W1 = tel_seq.shape
    _,  H2, W2 = maha_seq.shape
    maxH, maxW = max(H1, H2), max(W1, W2)
    tel_seq  = np.pad(tel_seq,  ((0,0),(0,maxH-H1),(0,maxW-W1)))
    maha_seq = np.pad(maha_seq, ((0,0),(0,maxH-H2),(0,maxW-W2)))
    data = np.concatenate([tel_seq, maha_seq], axis=1)[:, :256, :256]
    print(f"  Loaded shape: {data.shape}")

X_test = torch.from_numpy(data[:4]).float().unsqueeze(0).unsqueeze(2)  # (1,4,1,256,256)
y_test = torch.from_numpy(data[4]).float().unsqueeze(0).unsqueeze(0)   # (1,1,256,256)

model = ConvLSTMEncoderDecoder().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

with torch.no_grad():
    output = model(X_test.to(device))
print(f"Output shape : {output.shape}")
print(f"Output range : {output.min().item():.1f} – {output.max().item():.1f}")

📂 Normal mode: full numpy load...
  Loaded shape: (5, 256, 256)
Parameters: 776,705
Output shape : torch.Size([1, 1, 256, 256])
Output range : -0.3 – 0.1


In [5]:
# Cell 5 — Loss + Metrics
class PopulationLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3):
        super().__init__()
        self.alpha, self.beta = alpha, beta
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()

    def forward(self, pred, target):
        mse  = self.mse(pred, target)
        mae  = self.mae(pred, target)
        pc   = torch.clamp(pred,   min=0)
        tc   = torch.clamp(target, min=0)
        mask = tc > 1.0
        rel  = (torch.abs(pc[mask] - tc[mask]) / (tc[mask] + 1e-8)).mean() \
               if mask.sum() > 0 else torch.tensor(0.0, device=pred.device)
        return self.alpha * mse + self.beta * mae + 0.1 * rel

def calculate_r2(pred, target):
    ss_res = ((pred - target) ** 2).sum()
    ss_tot = ((target - target.mean()) ** 2).sum()
    return (1 - ss_res / ss_tot).item()

criterion = PopulationLoss()
print("✅ Model classes and loss function defined")


✅ Model classes and loss function defined


In [6]:
# Cell 6 — Save architecture
Path("models").mkdir(exist_ok=True)
torch.save(model.state_dict(), "models/model_architecture.pt")
print("✅ Saved: models/model_architecture.pt")
print("=" * 70)
print("NOTEBOOK 04 COMPLETE — Next: Notebook 05 Training")
print("=" * 70)

✅ Saved: models/model_architecture.pt
NOTEBOOK 04 COMPLETE — Next: Notebook 05 Training
